# Starter Tutorial
This page provides a guide for getting started running your first AI-powered research!

<i>The <a href="https://docs.expectedparrot.com" target="_blank" class="text-blue-500 hover:text-blue-600">EDSL Docs<i class="fa fa-mail-forward"></i></a> also provide installation instructions, starter tutorials, and in-depth documentation for the components of the EDSL library.</i>

In [1]:
# ! pip install edsl

## Part 1: Using API Keys

Large language models (LLMs) are at the heart of AI-powered research. EDSL allows you to easily conduct research with popular LLMs, including OpenAI's GPTs, Google's Gemini, Anthropic's Claude, Llama 2 and others. In order to do so, you must provide EDSL with your API keys from LLM providers. <i>EDSL will never store your personal API keys.</i>

There are 2 ways to provide your API keys to EDSL:

### 1. Recommended: Use a .env file

Create a file with the name `.env` in your working directory and populate it as follows, replacing `your_key_here` with your actual API keys:

```
   ANTHROPIC_API_KEY=your_key_here
   DEEP_INFRA_API_KEY=your_key_here
   GOOGLE_API_KEY=your_key_here
   OPENAI_API_KEY=your_key_here
```

EDSL will read your API keys from this file. This method allows you to avoid having to provide your keys each time that you start a session with EDSL and is the recommended way to provide your API keys.

If you are not planning to use one of these providers you can comment out the line.
This will ensure that you get an error message if you accidentally try to use that provider without a valid API key. For example, if you only plan to use models provided by Anthropic, you can set your `.env` file as follows:

```
   ANTHROPIC_API_KEY=your_key_here
   # DEEP_INFRA_API_KEY=your_key_here
   # GOOGLE_API_KEY=your_key_here
   # OPENAI_API_KEY=your_key_here
```

### 2. Set your API keys in your Python code

You can alternatively set your API keys in your Python code before you import any EDSL object as follows:

```
   import os
   os.environ['ANTHROPIC_API_KEY'] = 'your_key_here'
   os.environ['DEEP_INFRA_API_KEY'] = 'your_key_here'
   os.environ['GOOGLE_API_KEY'] = 'your_key_here'
   os.environ['OPENAI_API_KEY'] = 'your_key_here'
```

Note that EDSL will store your API keys in your system's memory only for the duration of your session. If you restart your session, you will need to set your API keys again.

If you are not planning to use one of these providers you can delete the line. This will ensure that you get an error message if you accidentally try to use that provider without a valid API key.

### Caution

You should always treat your API keys like you treat your passwords: never share them or upload files with your API keys to any public repository.


## Part 2: Conducting AI-powered research
With your API keys in place, you're now ready for AI-powered research! In this part we show how to create and run a simple survey in just a couple lines of code, and then create a proper survey with only slightly more work.

### Quickstart Example
Here we create a multiple choice question, prompt an AI agent answer it, and inspect the results:

In [2]:
# Import a desired question type
from edsl.questions import QuestionMultipleChoice

# Construct a question
q = QuestionMultipleChoice(
    question_name = "example_question",
    question_text = "How do you feel today?",
    question_options = ["Bad", "OK", "Good"]
)

# Prompt the default model to answer it (GPT-4)
results = q.run()

# Inspect the results
results.select("example_question").print()

┏━━━━━━━━━━━━━━━━━━━┓
┃ answer            ┃
┃ .example_question ┃
┡━━━━━━━━━━━━━━━━━━━┩
│ Good              │
└───────────────────┘

### A Proper Survey
Here we create a more complex survey where we ask AI agents how much they enjoy different activities. We also create agents with different personas, and use different LLMs to generate the results:

In [3]:
from edsl.questions import QuestionLinearScale, QuestionFreeText
from edsl import Scenario, Survey, Agent, Model

# Construct questions 
# Note that we use a parameter `activity` in order to create multiple scenarios of the question texts
q1 = QuestionLinearScale(
    question_name = "q1",
    question_text = "On a scale from 0 to 5, how much do you enjoy {{ activity }}?",
    question_options = [0,1,2,3,4,5]
)

q2 = QuestionFreeText(
    question_name = "q2",
    question_text = "Describe your habits with respect to {{ activity }}."
)

# Scenarios let us construct multiple variations of the same questions
activities = ["exercising", "reading", "cooking"]
scenarios = [Scenario({"activity": a}) for a in activities]

# Combine the questions in a survey
survey = Survey(questions = [q1, q2])

# Create personas for agents that will respond to the survey
personas = ["You are an athlete", "You are a student", "You are a chef"]
agents = [Agent(traits = {"persona": p}) for p in personas]

# Select language models
models = [Model("gpt-3.5-turbo"), Model("gpt-4-1106-preview")]

# Administer the survey 
results = survey.by(scenarios).by(agents).by(models).run()

# View the results
results.select("model.model", "scenario.activity", "agent.persona", "answer.*").print()

┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ model              ┃ scenario   ┃ agent              ┃ answer                 ┃ answer                 ┃ answer ┃
┃ .model             ┃ .activity  ┃ .persona           ┃ .q2                    ┃ .q1_comment            ┃ .q1    ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ gpt-3.5-turbo      │ exercising │ You are a student  │ I try to exercise at   │ I enjoy exercising     │ 3      │
│                    │            │                    │ least three times a    │ moderately, it helps   │        │
│                    │            │                    │ week, usually going    │ me stay healthy and    │        │
│                    │            │                    │ for a run or doing a   │ relieve stress.        │        │
│                    │            │                    │ workout at the gym. I  │                        │        │
│                    │            │                    │ find that it helps me  │                        │        │
│                    │            │                    │ stay energized and     │                        │        │
│                    │            │                    │ focused throughout the │                        │        │
│                    │            │                    │ day.                   │                        │        │
├────────────────────┼────────────┼────────────────────┼────────────────────────┼────────────────────────┼────────┤
│ gpt-3.5-turbo      │ exercising │ You are a chef     │ As a chef, my job      │ As a chef, I enjoy     │ 1      │
│                    │            │                    │ keeps me on my feet    │ cooking and creating   │        │
│                    │            │                    │ for long hours in the  │ delicious dishes more  │        │
│                    │            │                    │ kitchen, so I consider │ than exercising.       │        │
│                    │            │                    │ that my exercise. I    │                        │        │
│                    │            │                    │ also enjoy going for   │                        │        │
│                    │            │                    │ walks or bike rides on │                        │        │
│                    │            │                    │ my days off to stay    │                        │        │
│                    │            │                    │ active and clear my    │                        │        │
│                    │            │                    │ mind.                  │                        │        │
├────────────────────┼────────────┼────────────────────┼────────────────────────┼────────────────────────┼────────┤
│ gpt-3.5-turbo      │ cooking    │ You are a student  │ I enjoy cooking and    │ I enjoy cooking a      │ 2      │
│                    │            │                    │ like to experiment     │ moderate amount, it's  │        │
│                    │            │                    │ with new recipes. I    │ a fun way to relax and │        │
│                    │            │                    │ usually cook simple    │ be creative in the     │        │
│                    │            │                    │ meals during the week  │ kitchen.               │        │
│                    │            │                    │ and try more elaborate │                        │        │
│                    │            │                    │ dishes on weekends     │                        │        │
│                    │            │                    │ when I have more time. │                        │        │
│                    │            │                    │ Cooking is a way for   │                        │        │
│                    │            │                    │

Notice that agents answer the questions and also provide explanations for their answers.

### Exploring your results
You can use our built-in methods to for analyzing and visualizing your results. You can also export them as a Pandas dataframe:

In [4]:
# Turn the Results object to a pandas dataframe
results.to_pandas()

,agent.agent_name,agent.persona,answer.q1,answer.q1_comment,answer.q2,iteration.iteration,model.frequency_penalty,model.logprobs,model.max_tokens,model.model,...,model.temperature,model.top_logprobs,model.top_p,prompt.q1_system_prompt,prompt.q1_user_prompt,prompt.q2_system_prompt,prompt.q2_user_prompt,raw_model_response.q1_raw_model_response,raw_model_response.q2_raw_model_response,scenario.activity
0,Agent_1,You are a student,3,"I enjoy exercising moderately, it helps me sta...","I try to exercise at least three times a week,...",0,0,False,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1BFWiej2odqhesoQyInWN7wjj9...,{'id': 'chatcmpl-9CaVj6V9cObwIhyd2sGCn5CS1Pkk8...,exercising
1,Agent_2,You are a chef,1,"As a chef, I enjoy cooking and creating delici...","As a chef, my job keeps me on my feet for long...",0,0,False,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1B1JlDKcn3j74QXsGbB1k4TmNo...,{'id': 'chatcmpl-9CaVjB99PfttdgtQQrdtu5ZLeYVHw...,exercising
2,Agent_1,You are a student,2,"I enjoy cooking a moderate amount, it's a fun ...",I enjoy cooking and like to experiment with ne...,0,0,False,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1BCHU5EWmdq8njlSdfpWzFtpFW...,{'id': 'chatcmpl-9CaVj253qpsXzbwWhdz6RkZOdc3RJ...,cooking
3,Agent_3,You are an athlete,5,I enjoy exercising a lot. It helps me stay fit...,I exercise regularly to maintain my fitness le...,0,0,False,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1B4XjHTuAUbaL6wiIovdApOiR6...,{'id': 'chatcmpl-9CaVjLIP0kGyHrtpOEahkQ7hqjaIl...,exercising
4,Agent_2,You are a chef,5,I absolutely love cooking. It's my passion and...,"As a chef, I have developed a routine of plann...",0,0,False,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1BId8HMuj7u93iwcgkrcyGEsGg...,{'id': 'chatcmpl-9CaVkivSkXLpsKjSicJkUYngc9WoM...,cooking
5,Agent_2,You are a chef,5,"I enjoy reading a lot, it allows me to discove...","As a chef, I enjoy reading cookbooks, food mag...",0,0,False,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1BtnDbVTovhkc93n1AhgIQcmAT...,{'id': 'chatcmpl-9CaVkKHstxPRlZQHbAzfWzQbMGqsU...,reading
6,Agent_1,You are a student,4,"I really enjoy reading, it helps me relax and ...",I try to read for at least 30 minutes every da...,0,0,False,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1BSpP7wTw2XyXG22Wv9IMmVAOU...,{'id': 'chatcmpl-9CaVjGfBiiP5ptNlqbRdT84ui9PWn...,reading
7,Agent_3,You are an athlete,1,"I enjoy reading a little bit, so I'll choose o...","As an athlete, I prioritize reading materials ...",0,0,False,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text

In [5]:
# The Results object has various attributes you can use
results.columns

['agent.agent_name',
 'agent.persona',
 'answer.q1',
 'answer.q1_comment',
 'answer.q2',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.q1_system_prompt',
 'prompt.q1_user_prompt',
 'prompt.q2_system_prompt',
 'prompt.q2_user_prompt',
 'raw_model_response.q1_raw_model_response',
 'raw_model_response.q2_raw_model_response',
 'scenario.activity']

In [6]:
# The Results object also supports SQL-like queries
results.sql("select * from self", shape="wide")

,agent.agent_name,agent.persona,answer.q1,answer.q1_comment,answer.q2,iteration.iteration,model.frequency_penalty,model.logprobs,model.max_tokens,model.model,...,model.temperature,model.top_logprobs,model.top_p,prompt.q1_system_prompt,prompt.q1_user_prompt,prompt.q2_system_prompt,prompt.q2_user_prompt,raw_model_response.q1_raw_model_response,raw_model_response.q2_raw_model_response,scenario.activity
0,Agent_1,You are a student,3,"I enjoy exercising moderately, it helps me sta...","I try to exercise at least three times a week,...",0,0,0,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1BFWiej2odqhesoQyInWN7wjj9...,{'id': 'chatcmpl-9CaVj6V9cObwIhyd2sGCn5CS1Pkk8...,exercising
1,Agent_2,You are a chef,1,"As a chef, I enjoy cooking and creating delici...","As a chef, my job keeps me on my feet for long...",0,0,0,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1B1JlDKcn3j74QXsGbB1k4TmNo...,{'id': 'chatcmpl-9CaVjB99PfttdgtQQrdtu5ZLeYVHw...,exercising
2,Agent_1,You are a student,2,"I enjoy cooking a moderate amount, it's a fun ...",I enjoy cooking and like to experiment with ne...,0,0,0,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1BCHU5EWmdq8njlSdfpWzFtpFW...,{'id': 'chatcmpl-9CaVj253qpsXzbwWhdz6RkZOdc3RJ...,cooking
3,Agent_3,You are an athlete,5,I enjoy exercising a lot. It helps me stay fit...,I exercise regularly to maintain my fitness le...,0,0,0,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1B4XjHTuAUbaL6wiIovdApOiR6...,{'id': 'chatcmpl-9CaVjLIP0kGyHrtpOEahkQ7hqjaIl...,exercising
4,Agent_2,You are a chef,5,I absolutely love cooking. It's my passion and...,"As a chef, I have developed a routine of plann...",0,0,0,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1BId8HMuj7u93iwcgkrcyGEsGg...,{'id': 'chatcmpl-9CaVkivSkXLpsKjSicJkUYngc9WoM...,cooking
5,Agent_2,You are a chef,5,"I enjoy reading a lot, it allows me to discove...","As a chef, I enjoy reading cookbooks, food mag...",0,0,0,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1BtnDbVTovhkc93n1AhgIQcmAT...,{'id': 'chatcmpl-9CaVkKHstxPRlZQHbAzfWzQbMGqsU...,reading
6,Agent_1,You are a student,4,"I really enjoy reading, it helps me relax and ...",I try to read for at least 30 minutes every da...,0,0,0,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,"{'text': ""You are answering questions as if yo...",{'text': 'You are being asked the following qu...,{'id': 'chatcmpl-9Ca1BSpP7wTw2XyXG22Wv9IMmVAOU...,{'id': 'chatcmpl-9CaVjGfBiiP5ptNlqbRdT84ui9PWn...,reading
7,Agent_3,You are an athlete,1,"I enjoy reading a little bit, so I'll choose o...","As an athlete, I prioritize reading materials ...",0,0,0,1000,gpt-3.5-turbo,...,0.5,3,1,"{'text': ""You are answering questions

Learn more about use cases and ways to conduct AI-powered research in the <a href="http://localhost:8000/getting-started#edsl-showcase" target="_blank" class="text-blue-500 hover:text-blue-600">EDSL Showcase</a>.

---
<p style="font-size: 14px;">Copyright © 2024 Expected Parrot, Inc. All rights reserved.   <a href="www.expectedparrot.com" style="color:#130061">www.expectedparrot.com</a></p>